# Multi-point joint Bayesian Optimisation

In [1]:
import torch
from nubo.acquisition import MCExpectedImprovement, MCUpperConfidenceBound
from nubo.models import GaussianProcess, fit_gp
from nubo.optimisation import joint
from nubo.test_functions import Hartmann6D
from nubo.utils import LatinHypercubeSampling, unnormalise
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.mlls import ExactMarginalLogLikelihood


# test function
func = Hartmann6D(minimise=False)
dims = func.dims
bounds = func.bounds

# training data
torch.manual_seed(1)
lhs = LatinHypercubeSampling(dims=dims)
x_train = lhs.maximin(points=dims*5)
x_train = unnormalise(x_train, bounds=bounds)
y_train = func(x_train)

# Bayesian optimisation loop
iters = 10

for iter in range(iters):
    
    # specify Gaussian process
    likelihood = GaussianLikelihood()
    gp = GaussianProcess(x_train, y_train, likelihood=likelihood)
    mll = ExactMarginalLogLikelihood(likelihood=likelihood, model=gp)
    
    # fit Gaussian process
    fit_gp(x_train, y_train, gp=gp, likelihood=likelihood, mll=mll, lr=0.1, steps=200)

    # specify acquisition function
    # acq = MCExpectedImprovement(gp=gp, y_best=torch.max(y_train), samples=256)
    acq = MCUpperConfidenceBound(gp=gp, beta=1.96**2, samples=256)

    # optimise acquisition function
    x_new, _ = joint(func=acq, method="Adam", batch_size=4, bounds=bounds, lr=0.1, steps=200, num_starts=1)

    # evaluate new point
    y_new = func(x_new)
    
    # add to data
    x_train = torch.vstack((x_train, x_new))
    y_train = torch.hstack((y_train, y_new))

    # print new best
    if torch.max(y_new) > torch.max(y_train[:-y_new.size(0)]):
        best_eval = torch.argmax(y_train)
        print(f"New best at evaluation {best_eval+1}: \t Inputs: {x_train[best_eval, :].numpy().reshape(dims).round(4)}, \t Outputs: {-y_train[best_eval].numpy().round(4)}")

# results
best_iter = int(torch.argmax(y_train))
print(f"Evaluation: {best_iter+1} \t Solution: {float(y_train[best_iter]):.4f}")


New best at evaluation 33: 	 Inputs: [0.4206 0.9388 0.7492 0.4295 0.4485 0.0024], 	 Outputs: -2.4497
New best at evaluation 38: 	 Inputs: [0.4075 0.998  0.0014 0.5362 0.0043 0.0011], 	 Outputs: -2.676
New best at evaluation 45: 	 Inputs: [4.030e-01 8.936e-01 9.995e-01 5.419e-01 6.400e-03 6.000e-04], 	 Outputs: -3.0968
New best at evaluation 47: 	 Inputs: [4.081e-01 8.699e-01 9.953e-01 5.815e-01 9.000e-04 4.000e-04], 	 Outputs: -3.1251
New best at evaluation 61: 	 Inputs: [3.980e-01 8.873e-01 9.979e-01 5.819e-01 7.000e-03 1.000e-04], 	 Outputs: -3.1261
New best at evaluation 66: 	 Inputs: [0.4037 0.8744 0.5707 0.57   0.0075 0.0469], 	 Outputs: -3.1792
Evaluation: 66 	 Solution: 3.1792
